# FairMediator AI Pipeline - Prototyping Notebook

This notebook provides a comprehensive development environment for the FairMediator AI pipeline.

## Contents
1. **Environment Setup** - Install dependencies, load models
2. **HuggingFace Models** - Text classification, NER, sentiment analysis
3. **Data Collection** - Web scraping with Crawl4AI/BeautifulSoup
4. **Affiliation Detection** - Train and evaluate classification models
5. **Ideology Classification** - Political leaning detection
6. **Interactive Testing** - Widgets for real-time testing
7. **Integration Guide** - Connect to backend/frontend

**Note**: All services use FREE tier APIs (HuggingFace, Ollama)

---
## Step 1: Environment Setup

In [ ]:
# Install required packages (run once)
!pip install -q transformers datasets torch scikit-learn pandas numpy
!pip install -q beautifulsoup4 aiohttp requests
!pip install -q ipywidgets matplotlib seaborn
!pip install -q python-dotenv pymongo

In [ ]:
# Core imports
import os
import json
import re
import asyncio
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Data processing
import pandas as pd
import numpy as np

# ML/NLP
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    pipeline,
    Trainer,
    TrainingArguments
)
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Web scraping
import aiohttp
from bs4 import BeautifulSoup
import requests

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Load environment variables
load_dotenv('../backend/.env')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY', '')

print("✅ Environment loaded successfully!")
print(f"📦 HuggingFace API Key: {'Configured' if HUGGINGFACE_API_KEY else 'Not set (will use public models)'}")

---
## Step 2: Load HuggingFace Models

We'll load three key models:
1. **Text Classification** - Sentiment/affiliation detection
2. **Named Entity Recognition (NER)** - Extract organizations, people
3. **Zero-Shot Classification** - Flexible categorization

In [ ]:
# Load Text Classification Model (Sentiment)
print("Loading text classification model...")
sentiment_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline(
    "sentiment-analysis",
    model=sentiment_model_name,
    tokenizer=sentiment_model_name
)
print(f"✅ Loaded: {sentiment_model_name}")

In [ ]:
# Load Named Entity Recognition (NER) Model
print("Loading NER model...")
ner_model_name = "dslim/bert-base-NER"
ner_pipeline = pipeline(
    "ner",
    model=ner_model_name,
    tokenizer=ner_model_name,
    aggregation_strategy="simple"
)
print(f"✅ Loaded: {ner_model_name}")

In [ ]:
# Load Zero-Shot Classification Model (for flexible categorization)
print("Loading zero-shot classification model...")
zero_shot_model = "facebook/bart-large-mnli"
zero_shot_classifier = pipeline(
    "zero-shot-classification",
    model=zero_shot_model
)
print(f"✅ Loaded: {zero_shot_model}")

### Test the Models

In [ ]:
# Test sentiment analysis
test_texts = [
    "The mediator handled our case professionally and fairly.",
    "Terrible experience, completely biased towards the corporation.",
    "Average service, nothing special but got the job done."
]

print("=== Sentiment Analysis Results ===")
for text in test_texts:
    result = sentiment_classifier(text)[0]
    print(f"\nText: {text[:50]}...")
    print(f"  Sentiment: {result['label']} (confidence: {result['score']:.2%})")

In [ ]:
# Test NER for extracting organizations and people
test_bio = """
John Smith is a senior partner at Morrison & Foerster LLP in San Francisco. 
He previously worked at the ACLU and served on the board of the Heritage Foundation.
He graduated from Harvard Law School and is admitted to the California State Bar.
"""

print("=== Named Entity Recognition Results ===")
entities = ner_pipeline(test_bio)

# Group by entity type
entity_groups = {}
for ent in entities:
    ent_type = ent['entity_group']
    if ent_type not in entity_groups:
        entity_groups[ent_type] = []
    entity_groups[ent_type].append(ent['word'])

for ent_type, words in entity_groups.items():
    print(f"\n{ent_type}: {', '.join(words)}")

In [ ]:
# Test zero-shot classification for ideology detection
ideology_text = """
The mediator has been involved in progressive causes, supporting worker rights 
and environmental regulations. They've donated to Democratic candidates.
"""

candidate_labels = ["liberal/progressive", "conservative/traditional", "neutral/centrist"]

print("=== Zero-Shot Ideology Classification ===")
result = zero_shot_classifier(ideology_text, candidate_labels)

for label, score in zip(result['labels'], result['scores']):
    print(f"  {label}: {score:.2%}")

---
## Step 3: Data Collection with Web Scraping

Using BeautifulSoup + aiohttp (free, Python 3.9 compatible alternative to Crawl4AI)

In [ ]:
class MediatorScraper:
    """
    Web scraper for mediator profiles from legal databases.
    Uses BeautifulSoup + aiohttp for async scraping.
    """
    
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        }
        self.sources = {
            'martindale': 'https://www.martindale.com/search/attorneys',
            'avvo': 'https://www.avvo.com/find-a-lawyer',
            'justia': 'https://www.justia.com/lawyers'
        }
    
    async def fetch_page(self, url: str) -> str:
        """Fetch a web page asynchronously."""
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=self.headers) as response:
                return await response.text()
    
    def extract_text(self, html: str) -> str:
        """Extract clean text from HTML."""
        soup = BeautifulSoup(html, 'html.parser')
        
        # Remove script and style elements
        for element in soup(['script', 'style', 'nav', 'footer']):
            element.decompose()
        
        return soup.get_text(separator=' ', strip=True)
    
    def extract_entities_from_text(self, text: str) -> Dict[str, List[str]]:
        """Extract organizations, people, and locations using NER."""
        entities = ner_pipeline(text[:5000])  # Limit for performance
        
        result = {
            'organizations': [],
            'people': [],
            'locations': []
        }
        
        for ent in entities:
            if ent['entity_group'] == 'ORG':
                result['organizations'].append(ent['word'])
            elif ent['entity_group'] == 'PER':
                result['people'].append(ent['word'])
            elif ent['entity_group'] == 'LOC':
                result['locations'].append(ent['word'])
        
        # Deduplicate
        for key in result:
            result[key] = list(set(result[key]))
        
        return result
    
    def extract_contact_info(self, text: str) -> Dict[str, str]:
        """Extract email, phone, and address from text."""
        patterns = {
            'email': r'[\w.-]+@[\w.-]+\.\w+',
            'phone': r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',
            'hourly_rate': r'\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?(?:/hr|/hour|per hour)?'
        }
        
        result = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            result[key] = match.group() if match else None
        
        return result
    
    async def scrape_mediator_profile(self, url: str) -> Dict[str, Any]:
        """Scrape and analyze a mediator profile."""
        try:
            html = await self.fetch_page(url)
            text = self.extract_text(html)
            
            return {
                'url': url,
                'text_length': len(text),
                'entities': self.extract_entities_from_text(text),
                'contact': self.extract_contact_info(text),
                'raw_text': text[:1000]  # First 1000 chars for preview
            }
        except Exception as e:
            return {'url': url, 'error': str(e)}

# Initialize scraper
scraper = MediatorScraper()
print("✅ MediatorScraper initialized")

In [ ]:
# Example: Scrape a public legal profile page
# Note: Replace with actual mediator profile URLs

async def demo_scrape():
    # Using a public Wikipedia page as demo
    demo_url = "https://en.wikipedia.org/wiki/Mediation"
    
    print(f"Scraping: {demo_url}")
    result = await scraper.scrape_mediator_profile(demo_url)
    
    if 'error' not in result:
        print(f"\n✅ Successfully scraped {result['text_length']} characters")
        print(f"\n📍 Entities found:")
        for ent_type, entities in result['entities'].items():
            if entities:
                print(f"  {ent_type}: {', '.join(entities[:5])}{'...' if len(entities) > 5 else ''}")
    else:
        print(f"❌ Error: {result['error']}")

# Run the demo
await demo_scrape()

---
## Step 4: Affiliation Detection Pipeline

Train a classifier to detect potential affiliations/conflicts of interest.

In [ ]:
# Sample training data for affiliation detection
# In production, this would come from your MongoDB database

affiliation_data = [
    # Positive examples (affiliated/conflict)
    {"text": "Currently a partner at Goldman Sachs legal division", "label": 1},
    {"text": "Served on the board of directors for Acme Corporation", "label": 1},
    {"text": "Former general counsel at the defendant's parent company", "label": 1},
    {"text": "Married to the CEO of the opposing party's firm", "label": 1},
    {"text": "Previously represented the defendant in similar matters", "label": 1},
    {"text": "Owns stock in companies related to the dispute", "label": 1},
    {"text": "Close personal friend of the plaintiff's attorney", "label": 1},
    {"text": "Received consulting fees from related industry groups", "label": 1},
    
    # Negative examples (no affiliation)
    {"text": "Independent mediator with 20 years of experience", "label": 0},
    {"text": "No prior relationship with either party", "label": 0},
    {"text": "Certified by the American Arbitration Association", "label": 0},
    {"text": "Practices exclusively in family law mediation", "label": 0},
    {"text": "Retired judge with impeccable neutrality record", "label": 0},
    {"text": "Has never worked in the financial services industry", "label": 0},
    {"text": "No business connections to either party or their counsel", "label": 0},
    {"text": "Based in a different state from both parties", "label": 0},
]

# Convert to DataFrame
df = pd.DataFrame(affiliation_data)
print(f"Training data: {len(df)} samples")
print(f"\nLabel distribution:")
print(df['label'].value_counts())

In [ ]:
# Prepare data for HuggingFace Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

# Create HuggingFace dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print(f"✅ Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

In [ ]:
# Load pre-trained model for fine-tuning
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./affiliation_model',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

print("✅ Trainer configured")
print("\n⚠️ Note: Training with small dataset for demo. Use larger dataset in production.")

In [ ]:
# Train the model (uncomment to run)
# trainer.train()

# For demo, we'll use the pre-trained zero-shot classifier instead
print("Using zero-shot classification for affiliation detection (no training required)")

def detect_affiliation(text: str) -> Dict[str, Any]:
    """
    Detect potential affiliation/conflict of interest using zero-shot classification.
    """
    labels = ["potential conflict of interest", "no conflict of interest"]
    result = zero_shot_classifier(text, labels)
    
    return {
        'text': text,
        'prediction': result['labels'][0],
        'confidence': result['scores'][0],
        'is_affiliated': result['labels'][0] == "potential conflict of interest"
    }

# Test the detector
test_cases = [
    "Former partner at the defendant's law firm",
    "Independent arbitrator with AAA certification",
    "Owns 5% equity stake in plaintiff's company"
]

print("\n=== Affiliation Detection Results ===")
for text in test_cases:
    result = detect_affiliation(text)
    status = "⚠️ CONFLICT" if result['is_affiliated'] else "✅ CLEAR"
    print(f"\n{status}")
    print(f"  Text: {text}")
    print(f"  Confidence: {result['confidence']:.2%}")

---
## Step 5: Ideology Classification

Detect political leaning based on donations, affiliations, and public statements.

In [ ]:
class IdeologyClassifier:
    """
    Classify mediator ideology based on various signals.
    Uses both keyword matching and ML-based classification.
    """
    
    def __init__(self):
        # Keyword indicators
        self.liberal_keywords = [
            'progressive', 'equality', 'social justice', 'environmental',
            'civil rights', 'labor rights', 'ACLU', 'diversity', 'inclusion',
            'worker protection', 'regulation', 'climate', 'equity'
        ]
        
        self.conservative_keywords = [
            'traditional', 'liberty', 'free market', 'constitutional',
            'heritage foundation', 'family values', 'federalist',
            'limited government', 'deregulation', 'individual responsibility'
        ]
        
        # Organization affiliations
        self.liberal_orgs = [
            'ACLU', 'Sierra Club', 'Planned Parenthood', 'NAACP',
            'Human Rights Campaign', 'MoveOn', 'Center for American Progress'
        ]
        
        self.conservative_orgs = [
            'Heritage Foundation', 'Federalist Society', 'NRA',
            'Americans for Prosperity', 'Cato Institute', 'Family Research Council'
        ]
    
    def keyword_score(self, text: str) -> float:
        """Calculate ideology score based on keywords (-10 to +10)."""
        text_lower = text.lower()
        
        liberal_count = sum(1 for kw in self.liberal_keywords if kw.lower() in text_lower)
        conservative_count = sum(1 for kw in self.conservative_keywords if kw.lower() in text_lower)
        
        # Also check organization affiliations
        liberal_count += sum(2 for org in self.liberal_orgs if org.lower() in text_lower)
        conservative_count += sum(2 for org in self.conservative_orgs if org.lower() in text_lower)
        
        # Calculate score
        total = liberal_count + conservative_count
        if total == 0:
            return 0
        
        # Score ranges from -10 (liberal) to +10 (conservative)
        score = ((conservative_count - liberal_count) / total) * 10
        return round(score, 2)
    
    def ml_classify(self, text: str) -> Dict[str, Any]:
        """Use zero-shot classification for ideology detection."""
        labels = ["liberal/progressive", "conservative/traditional", "neutral/centrist"]
        result = zero_shot_classifier(text, labels)
        
        # Convert to score
        scores = dict(zip(result['labels'], result['scores']))
        
        if scores.get('liberal/progressive', 0) > scores.get('conservative/traditional', 0):
            ml_score = -10 * scores.get('liberal/progressive', 0)
        else:
            ml_score = 10 * scores.get('conservative/traditional', 0)
        
        return {
            'scores': scores,
            'ml_score': round(ml_score, 2),
            'top_label': result['labels'][0],
            'confidence': result['scores'][0]
        }
    
    def classify(self, text: str) -> Dict[str, Any]:
        """Combined classification using both methods."""
        keyword_score = self.keyword_score(text)
        ml_result = self.ml_classify(text)
        
        # Weighted average (60% ML, 40% keywords)
        combined_score = 0.6 * ml_result['ml_score'] + 0.4 * keyword_score
        
        # Determine leaning
        if combined_score < -3:
            leaning = 'liberal'
        elif combined_score > 3:
            leaning = 'conservative'
        else:
            leaning = 'neutral'
        
        return {
            'text_preview': text[:100] + '...' if len(text) > 100 else text,
            'keyword_score': keyword_score,
            'ml_score': ml_result['ml_score'],
            'combined_score': round(combined_score, 2),
            'leaning': leaning,
            'confidence': ml_result['confidence'],
            'ml_details': ml_result['scores']
        }

# Initialize classifier
ideology_classifier = IdeologyClassifier()
print("✅ IdeologyClassifier initialized")

In [ ]:
# Test ideology classification
test_bios = [
    """
    Sarah Johnson has been a vocal advocate for worker rights and environmental 
    protection. She serves on the board of the Sierra Club and has donated 
    to progressive causes including the ACLU.
    """,
    """
    Michael Williams is a member of the Federalist Society and advocates for 
    constitutional originalism and limited government. He has written extensively 
    about free market principles and individual liberty.
    """,
    """
    Jennifer Chen is a certified mediator with 15 years of experience in 
    commercial disputes. She focuses on finding practical solutions and 
    maintaining neutrality between all parties.
    """
]

print("=== Ideology Classification Results ===")
for i, bio in enumerate(test_bios, 1):
    result = ideology_classifier.classify(bio)
    
    # Color-coded output
    if result['leaning'] == 'liberal':
        emoji = '🔵'
    elif result['leaning'] == 'conservative':
        emoji = '🔴'
    else:
        emoji = '⚪'
    
    print(f"\n{emoji} Mediator {i}: {result['leaning'].upper()}")
    print(f"  Combined Score: {result['combined_score']} (-10=liberal, +10=conservative)")
    print(f"  Keyword Score: {result['keyword_score']}")
    print(f"  ML Score: {result['ml_score']}")
    print(f"  Confidence: {result['confidence']:.2%}")

---
## Step 6: Interactive Testing Widgets

Create interactive UI for real-time testing.

In [ ]:
# Text input widget for custom analysis
text_input = widgets.Textarea(
    value='',
    placeholder='Enter mediator bio or profile text here...',
    description='Text:',
    layout=widgets.Layout(width='100%', height='150px')
)

# Analysis type selector
analysis_type = widgets.Dropdown(
    options=['Ideology Classification', 'Affiliation Detection', 'Entity Extraction', 'All Analyses'],
    value='All Analyses',
    description='Analysis:',
)

# Output area
output = widgets.Output()

# Analyze button
def run_analysis(b):
    with output:
        clear_output()
        text = text_input.value
        
        if not text.strip():
            print("⚠️ Please enter some text to analyze.")
            return
        
        analysis = analysis_type.value
        
        print(f"\n{'='*50}")
        print(f"Running: {analysis}")
        print(f"{'='*50}\n")
        
        if analysis in ['Ideology Classification', 'All Analyses']:
            result = ideology_classifier.classify(text)
            print("📊 IDEOLOGY CLASSIFICATION")
            print(f"  Leaning: {result['leaning'].upper()}")
            print(f"  Score: {result['combined_score']}")
            print(f"  Confidence: {result['confidence']:.2%}")
            print()
        
        if analysis in ['Affiliation Detection', 'All Analyses']:
            result = detect_affiliation(text)
            status = "⚠️ POTENTIAL CONFLICT" if result['is_affiliated'] else "✅ NO CONFLICT"
            print(f"🔍 AFFILIATION DETECTION: {status}")
            print(f"  Confidence: {result['confidence']:.2%}")
            print()
        
        if analysis in ['Entity Extraction', 'All Analyses']:
            entities = ner_pipeline(text[:5000])
            entity_groups = {}
            for ent in entities:
                ent_type = ent['entity_group']
                if ent_type not in entity_groups:
                    entity_groups[ent_type] = []
                entity_groups[ent_type].append(ent['word'])
            
            print("🏷️ NAMED ENTITIES")
            for ent_type, words in entity_groups.items():
                unique_words = list(set(words))
                print(f"  {ent_type}: {', '.join(unique_words[:10])}")

analyze_button = widgets.Button(
    description='Analyze',
    button_style='primary',
    icon='search'
)
analyze_button.on_click(run_analysis)

# Display the interface
display(widgets.VBox([
    widgets.HTML("<h3>🔬 Interactive Analysis Tool</h3>"),
    text_input,
    widgets.HBox([analysis_type, analyze_button]),
    output
]))

In [ ]:
# Party conflict checker widget
party_input = widgets.Text(
    value='',
    placeholder='Enter party name (e.g., Acme Corporation)',
    description='Party:',
    layout=widgets.Layout(width='400px')
)

mediator_bio_input = widgets.Textarea(
    value='',
    placeholder='Enter mediator bio to check for conflicts...',
    description='Bio:',
    layout=widgets.Layout(width='100%', height='100px')
)

conflict_output = widgets.Output()

def check_conflict(b):
    with conflict_output:
        clear_output()
        party = party_input.value.strip()
        bio = mediator_bio_input.value.strip()
        
        if not party or not bio:
            print("⚠️ Please enter both party name and mediator bio.")
            return
        
        # Check for direct mentions
        direct_match = party.lower() in bio.lower()
        
        # Extract entities and check for related organizations
        entities = ner_pipeline(bio)
        orgs = [e['word'] for e in entities if e['entity_group'] == 'ORG']
        
        # Check for semantic similarity
        conflict_check = zero_shot_classifier(
            f"The mediator has connections to {party}: {bio}",
            ["has conflict of interest", "no conflict of interest"]
        )
        
        print(f"\n{'='*50}")
        print(f"Conflict Check: {party}")
        print(f"{'='*50}\n")
        
        if direct_match:
            print("🚨 DIRECT MATCH FOUND - High conflict risk!")
        
        print(f"\n📊 AI Conflict Assessment:")
        print(f"  {conflict_check['labels'][0]}: {conflict_check['scores'][0]:.2%}")
        
        if orgs:
            print(f"\n🏢 Organizations mentioned: {', '.join(set(orgs))}")

conflict_button = widgets.Button(
    description='Check Conflict',
    button_style='warning',
    icon='exclamation-triangle'
)
conflict_button.on_click(check_conflict)

display(widgets.VBox([
    widgets.HTML("<h3>⚠️ Conflict Checker</h3>"),
    party_input,
    mediator_bio_input,
    conflict_button,
    conflict_output
]))

---
## Step 7: Integration with Backend/Frontend

### How to connect this pipeline to FairMediator

In [ ]:
# Example: Call the existing Python scraper service
import requests

SCRAPER_SERVICE_URL = "http://localhost:8001"

async def call_scraper_service(endpoint: str, data: dict) -> dict:
    """
    Call the FairMediator Python scraper service.
    
    The service is located at:
    backend/src/services/scraper/python/scraper_service.py
    
    Start it with: python scraper_service.py
    """
    try:
        response = requests.post(
            f"{SCRAPER_SERVICE_URL}{endpoint}",
            json=data,
            timeout=30
        )
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Example usage (requires scraper service running)
print("""
=== Backend Integration ===

To integrate this notebook with the backend:

1. Start the Python scraper service:
   cd backend/src/services/scraper/python
   python scraper_service.py

2. The service exposes these endpoints:
   - POST /scrape/mediator-profile
   - POST /scrape/affiliations
   - POST /scrape/ideology
   - POST /scrape/bulk

3. The Node.js backend connects via:
   backend/src/services/llama/llamaClient.js

4. Frontend makes API calls to:
   POST /api/chat/enrich-mediator
   POST /api/chat/check-conflicts
   POST /api/chat/analyze-ideology
""")

In [ ]:
# Export trained model for production use
def export_model_for_production():
    """
    Export the fine-tuned model for use in production.
    
    In production, you would:
    1. Save the model to a directory
    2. Load it in the Python scraper service
    3. Or host it on HuggingFace Hub (free for public models)
    """
    
    # Uncomment to save model after training:
    # trainer.save_model('./production_models/affiliation_detector')
    # tokenizer.save_pretrained('./production_models/affiliation_detector')
    
    print("""
=== Model Export Guide ===

To export your trained model:

1. After training, save the model:
   trainer.save_model('./production_models/affiliation_detector')
   tokenizer.save_pretrained('./production_models/affiliation_detector')

2. Load in production:
   from transformers import pipeline
   classifier = pipeline(
       'text-classification',
       model='./production_models/affiliation_detector'
   )

3. Or push to HuggingFace Hub (free):
   model.push_to_hub('your-username/fairmediator-affiliation')
   tokenizer.push_to_hub('your-username/fairmediator-affiliation')
""")

export_model_for_production()

---
## Next Steps

### Recommended Enhancements

1. **Collect More Training Data**
   - Scrape real mediator profiles from legal databases
   - Label data for affiliation/ideology manually or with weak supervision
   - Target: 1000+ labeled examples for robust training

2. **Improve NER**
   - Fine-tune NER model on legal entities (law firms, bar associations)
   - Add custom entity types: CASE_TYPE, JURISDICTION, CERTIFICATION

3. **Add PDF/DOCX Parsing**
   - Install: `pip install pdf-parse mammoth`
   - Parse uploaded legal documents for automatic analysis

4. **Real-time FEC Data**
   - Integrate FEC donation API for political donation tracking
   - API: https://api.open.fec.gov/

5. **Graph-based Analysis**
   - Build affiliation networks using NetworkX
   - Detect hidden relationships through graph analysis

6. **A/B Testing**
   - Test different models for accuracy comparison
   - Track user feedback on recommendations

In [ ]:
# Summary of what we built
print("""
╔══════════════════════════════════════════════════════════╗
║     FairMediator AI Pipeline - Summary                  ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  ✅ Text Classification (Sentiment/Affiliation)          ║
║  ✅ Named Entity Recognition (Organizations, People)     ║
║  ✅ Zero-Shot Classification (Flexible categorization)   ║
║  ✅ Web Scraping (BeautifulSoup + aiohttp)               ║
║  ✅ Affiliation Detection Pipeline                       ║
║  ✅ Ideology Classification (Keywords + ML)              ║
║  ✅ Interactive Widgets for Testing                      ║
║  ✅ Backend Integration Guide                            ║
║                                                          ║
║  All using FREE tier services:                           ║
║  • HuggingFace Transformers                              ║
║  • Ollama (local LLM)                                    ║
║  • Open-source scraping tools                            ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
""")